<div align="center">
  <a href="https://github.com/andreihar/quilt-tex">
	<img src="readme/logo.svg" alt="Logo" width="105" height="80">
  </a>
  <h1>QuiltTex</h1>
</div>

## Texture synthesis

The texture synthesis process is based on the work by Efros and Freeman titled ["Image Quilting for Texture Synthesis and Transfer."](http://graphics.cs.cmu.edu/people/efros/research/quilting/quilting.pdf) The goal of texture synthesis is to generate a larger texture image from a small sample of an existing texture, ensuring that the synthesised texture looks natural and continuous without obvious seams or repetitions.

These texture synthesis approaches can be applied to various types of textures, whether structured (like bricks or tiles) or stochastic (like grass or sand). The choice of method and parameters, such as patch size and overlap width, depends on the specific characteristics of the input texture. For instance, highly structured textures may benefit more from Method 3 due to its ability to handle complex patterns with minimal visible seams.

In [ ]:
import numpy as np
import math
from skimage import io, util, color, img_as_ubyte
import heapq

### Random Patch Selection

This is the simplest approach, where patches are randomly selected from the input texture and placed into the new image. Because the patches are chosen randomly without considering the overlap with adjacent patches, noticeable edges or seams are likely to appear in the synthesised texture. This can make the final image look disjointed or artificial. 

In [ ]:
def randomPatch(texture, patchSize):
    h, w = texture.shape[:2]
    i = np.random.randint(h - patchSize)
    j = np.random.randint(w - patchSize)
    return texture[i:i+patchSize, j:j+patchSize]

def method1(texture, patchSize):
    texture = util.img_as_float(texture)
    h, w = texture.shape[:2]

    if len(texture.shape) == 2:
        texture = np.stack([texture]*3, axis=-1)

    patchH = math.ceil((5 * h) / patchSize)
    patchW = math.ceil((5 * w) / patchSize)

    output = np.zeros((patchH * patchSize, patchW * patchSize, texture.shape[2]))

    for i in range(patchH):
        for j in range(patchW):
            y = i * patchSize
            x = j * patchSize
            patch = randomPatch(texture, patchSize)
            output[y:y+patchSize, x:x+patchSize] = patch
    return output[:5*h, :5*w]

### Overlap-Constrained Patch Selection

In this method, the algorithm selects patches that best match the existing content in the overlapping region, based on a similarity measure Sum of Squared Differences. Instead of just picking the patch with the absolute lowest error, the algorithm introduces some randomness by selecting from among the patches that have errors within a certain tolerance of the minimum error.

This method reduces the likelihood of visible seams by ensuring that the overlapping regions of adjacent patches are more closely matched. The slight randomness helps avoid the problem of repetitive patterns, creating a more natural texture.

In [ ]:
def ssd(patch, patchSize, overlap, output, x, y):
    return (np.sum((patch[:, :overlap] - output[y:y+patchSize, x:x+overlap])**2) if x > 0 else 0) + \
           (np.sum((patch[:overlap, :] - output[y:y+overlap, x:x+patchSize])**2) if y > 0 else 0) - \
           (np.sum((patch[:overlap, :overlap] - output[y:y+overlap, x:x+overlap])**2) if x > 0 and y > 0 else 0) 

def bestPatch(texture, patchSize, overlap, output, x, y):
    h, w = texture.shape[:2]
    errors = np.array([[ssd(texture[i:i+patchSize, j:j+patchSize], patchSize, overlap, output, x, y) for j in range(w - patchSize)] for i in range(h - patchSize)])
    i, j = np.unravel_index(np.argmin(errors), errors.shape)
    return texture[i:i+patchSize, j:j+patchSize]

def method2(texture, patchSize, overlapSize):
    texture = util.img_as_float(texture)
    h, w = texture.shape[:2]

    patchH = math.ceil((5 * h - patchSize) / (patchSize - overlapSize)) + 1
    patchW = math.ceil((5 * w - patchSize) / (patchSize - overlapSize)) + 1

    if len(texture.shape) == 2:
        texture = np.stack([texture]*3, axis=-1)

    output = np.zeros(((patchH * patchSize) - (patchH - 1) * overlapSize, (patchW * patchSize) - (patchW - 1) * overlapSize, texture.shape[2]))

    for i in range(patchH):
        for j in range(patchW):
            y = i * (patchSize - overlapSize)
            x = j * (patchSize - overlapSize)

            if i == 0 and j == 0:
                patch = randomPatch(texture, patchSize)
            else:
                patch = bestPatch(texture, patchSize, overlapSize, output, x, y)
            output[y:y+patchSize, x:x+patchSize] = patch
    return output[:5*h, :5*w]

### Minimum Error Boundary Cut

This method refines the overlap between patches even further by calculating the optimal boundary within the overlapping region. The algorithm computes an "energy matrix" representing the difference between overlapping patches and then finds a path through this matrix that minimises the error. This path becomes the boundary where the new patch is blended into the existing texture.

By allowing the patches to have irregular, "ragged" edges, this method effectively minimises visible seams, even for highly structured textures. The dynamic programming approach used to find the minimum error cut ensures that the transition between patches is as seamless as possible.

In [ ]:
### Method 3
def cutPatch(patch, overlapSize, output, x, y):
    def path(errors):
        h, w = errors.shape
        queue = [(error, [i]) for i,error in enumerate(errors[0])]
        heapq.heapify(queue)
        visited = set()
        while queue:
            error, path = heapq.heappop(queue)
            depth = len(path)
            index = path[-1]
            if depth == h:
                return path
            for delta in -1, 0, 1:
                next = index + delta
                if 0 <= next < w:
                    if (depth, next) not in visited:
                        heapq.heappush(queue, (error + errors[depth, next], path + [next]))
                        visited.add((depth, next))

    patch = patch.copy()
    dy, dx = patch.shape[:2]
    cut = np.zeros_like(patch, dtype=bool)
    if x > 0:
        for i, j in enumerate(path(np.sum((patch[:,:overlapSize]-output[y:y+dy,x:x+overlapSize])**2, axis=2))):
            cut[i, :j] = True
    if y > 0:
        for j, i in enumerate(path(np.sum((patch[:overlapSize,:]-output[y:y+overlapSize,x:x+dx])**2, axis=2).T)):
            cut[:i, j] = True

    np.copyto(patch, output[y:y+dy, x:x+dx], where=cut)
    return patch

def method3(texture, patchSize, overlapSize):
    texture = util.img_as_float(texture)
    h, w = texture.shape[:2]

    patchH = math.ceil((5 * h - patchSize) / (patchSize - overlapSize)) + 1
    patchW = math.ceil((5 * w - patchSize) / (patchSize - overlapSize)) + 1

    if len(texture.shape) == 2:
        texture = np.stack([texture]*3, axis=-1)

    output = np.zeros(((patchH * patchSize) - (patchH - 1) * overlapSize, (patchW * patchSize) - (patchW - 1) * overlapSize, texture.shape[2]))

    for i in range(patchH):
        for j in range(patchW):
            y = i * (patchSize - overlapSize)
            x = j * (patchSize - overlapSize)

            if i == 0 and j == 0:
                patch = randomPatch(texture, patchSize)
            else:
                patch = bestPatch(texture, patchSize, overlapSize, output, x, y)
                patch = cutPatch(patch, overlapSize, output, x, y)
            output[y:y+patchSize, x:x+patchSize] = patch
    return output[:5*h, :5*w]

### Comparison

* **Random Patch Selection** is the simplest and fastest but often produces unsatisfactory results due to visible seams and a lack of coherence between patches.
* **Overlap-Constrained Patch Selection** improves on this by selecting patches that better align with the existing texture, reducing visible artifacts in the overlap regions.
* **Minimum Error Boundary Cut** offers the highest quality results by not only selecting well-matching patches but also optimising the boundary where these patches join, leading to a more seamless and natural appearance, but runs the longest.

In [ ]:
def save_image(output, filename):
    io.imshow(output)
    io.show()
    if len(output.shape) == 3:
        io.imsave(filename, img_as_ubyte(output[:,:,:3]))
    else:
        io.imsave(filename, img_as_ubyte(output))

In [ ]:
dataDir = 'data/textures/';
outDir = 'results_transfer/';

patchSize = 40
overlapErr = 7

fileName = 'toast.png'
texture = io.imread(dataDir + fileName)

save_image(method1(texture, patchSize), outDir + fileName.split('.')[0] + "_1.jpg")
save_image(method2(texture, patchSize, overlapErr), outDir + fileName.split('.')[0] + "_2.jpg")
save_image(method3(texture, patchSize, overlapErr), outDir + fileName.split('.')[0] + "_3.jpg")

## Texture transfer

Texture transfer is a technique that re-renders an image by applying the texture of one image onto the structure of another. This process blends the patterns of the source texture with the underlying structure of the target image, creating a unique visual effect where the content of the target image appears to be made up of the source texture.

The process works by balancing two main constraints:

* **Texture Consistency**: Ensuring that each patch from the source texture fits seamlessly with the previously synthesised parts of the new image. This ensures that the overall texture looks natural and continuous.
* **Structural Correspondence**: Each texture patch must also align with the features of the target image. This is achieved by matching the source texture to a map of the target image's grayscale intensity. The resulting image retains the visual structure of the target while taking on the texture of the source.

To balance these constraints, an error metric combines texture matching with structural alignment, controlled by a parameter `alpha` that dictates the trade-off between texture fidelity and adherence to the target image's features. The image is processed in a single pass, with patches selected to simultaneously match the texture and align with the target image's structure.

This technique enables the creation of striking visual effects, like rendering a photograph with a different material's texture, producing an image that is both visually rich and true to the original structure.

In [ ]:
def corrOver(texture, corrTex, corrTar, patchSize, overlap, output, x, y, alpha=0.1):
    h, w = texture.shape[:2]
    errors = np.zeros((h - patchSize, w - patchSize))
    corrTarP = corrTar[y:y+patchSize, x:x+patchSize]
    di, dj = corrTarP.shape
    for i in range(h - patchSize):
        for j in range(w - patchSize):
            errors[i, j] = alpha * (np.sum(ssd(texture[i:i+di, j:j+dj], patchSize, overlap, output, x, y))) + (1 - alpha) * np.sum((corrTex[i:i+di, j:j+dj] - corrTarP)**2)
    i, j = np.unravel_index(np.argmin(errors), errors.shape)
    return texture[i:i+di, j:j+dj]

def transfer(texture, target, patchSize, overlap, alpha):
    if texture.shape[2] == 4:
        texture = color.rgba2rgb(texture)
    if target.shape[2] == 4:
        target = color.rgba2rgb(target)
    corrTex = color.rgb2gray(texture)
    corrTar  = color.rgb2gray(target)
    texture = util.img_as_float(texture)[:,:,:3]
    target = util.img_as_float(target)[:,:,:3]
    h, w = target.shape[:2]
    output = np.zeros_like(target)
    for i in range((math.ceil((h - patchSize) / (patchSize - overlap)) + 1 or 1)):
        for j in range((math.ceil((w - patchSize) / (patchSize - overlap)) + 1 or 1)):
            y = i * (patchSize - overlap)
            x = j * (patchSize - overlap)
            if i == 0 and j == 0:
                patch = corrOver(texture, corrTex, corrTar, patchSize, overlap, output, x, y, alpha)
            else:
                patch = corrOver(texture, corrTex, corrTar, patchSize, overlap, output, x, y, alpha)
                patch = cutPatch(patch, overlap, output, x, y)
            output[y:y+patchSize, x:x+patchSize] = patch
    return output

In [ ]:
textDir = 'data/textures/';
imgDir = 'data/images/'
outDir = 'results_transfer/';

patchSize = 40
overlapErr = 7
alpha = 0.3

imageFile = 'keane.jpg'
texture = 'toast.png'

fileName = texture
texture = io.imread(textDir + fileName)
image = io.imread(imgDir + imageFile)
    
output = transfer(texture, image, patchSize, overlapErr, alpha)
save_image(output, outDir + fileName.split('.')[0] + "_" + imageFile.split('.')[0] + ".jpg")